In [ ]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

In [ ]:
# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

In [ ]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [ ]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-1209b9ad90a5-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/mount_name/topics/1209b9ad90a5.geo/partition=0/"
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

In [ ]:
dbutils.fs.ls("/mnt/mount_name/topics/1209b9ad90a5.geo/partition=0/topics/1209b9ad90a5.geo/partition=0/")

In [ ]:
%sql
-- Disable format checks during the reading of Delta tables
SET spark.databricks.delta.formatCheck.enabled=false

In [ ]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/mount_name/topics/1209b9ad90a5.geo/partition=0/topics/1209b9ad90a5.geo/partition=0/*.json" 
file_type = "json"

# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_geo = spark.read.format(file_type).option("inferSchema", infer_schema).load(file_location)
# Display Spark dataframe to check its content
display(df_geo)

In [ ]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/mount_name/topics/1209b9ad90a5.geo/partition=0/topics/1209b9ad90a5.pin/partition=0/*.json" 
file_type = "json"

# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_pin = spark.read.format(file_type).option("inferSchema", infer_schema).load(file_location)
# Display Spark dataframe to check its content
display(df_pin)

In [ ]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/mount_name/topics/1209b9ad90a5.geo/partition=0/topics/1209b9ad90a5.user/partition=0/*.json" 
file_type = "json"

# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_user = spark.read.format(file_type).option("inferSchema", infer_schema).load(file_location)
# Display Spark dataframe to check its content
display(df_user)

To clean the df_pin DataFrame you should perform the following transformations:

Replace empty entries and entries with no relevant data in each column with Nones
Perform the necessary transformations on the follower_count to ensure every entry is a number. Make sure the data type of this column is an int.
Ensure that each column containing numeric data has a numeric data type
Clean the data in the save_location column to include only the save location path
Rename the index column to ind.
Reorder the DataFrame columns to have the following column order:
ind
unique_id
title
description
follower_count
poster_name
tag_list
is_image_or_video
image_src
save_location
category

In [ ]:
from pyspark.sql.functions import col, regexp_replace, expr



# Replace empty entries and entries with no relevant data with Nones
df_pin_cleaned = df_pin.na.replace('', None)

# Perform transformations on follower_count column
df_pin_cleaned = df_pin_cleaned.withColumn("follower_count", expr(
    "CASE WHEN follower_count LIKE '%k' THEN CAST(regexp_replace(follower_count, 'k', '') AS DOUBLE) * 1000 " +
    "ELSE CAST(follower_count AS DOUBLE) END"
).cast("int"))

# Ensure numeric data columns have numeric data type
numeric_columns = ["follower_count"]  # Add other numeric columns if needed
for column in numeric_columns:
    df_pin_cleaned = df_pin_cleaned.withColumn(column, col(column).cast("double"))

# Clean data in save_location column to include only the save location path
df_pin_cleaned = df_pin_cleaned.withColumn("save_location", col("save_location").substr(23, 200))  # Assuming 23 is the starting index

df_pin_cleaned = df_pin_cleaned.na.replace('', None)

# Rename the index column to 'ind'
df_pin_cleaned = df_pin_cleaned.withColumnRenamed("index", "ind")

# Reorder DataFrame columns
column_order = ["ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category"]
df_pin_cleaned = df_pin_cleaned.select(column_order)

# Show the cleaned DataFrame
df_pin_cleaned.show()

clean the df_geo DataFrame you should perform the following transformations:

Create a new column coordinates that contains an array based on the latitude and longitude columns
Drop the latitude and longitude columns from the DataFrame
Convert the timestamp column from a string to a timestamp data type
Reorder the DataFrame columns to have the following column order:
ind
country
coordinates
timestamp

In [ ]:
from pyspark.sql.functions import col, array, to_timestamp

# Create a new column 'coordinates' based on latitude and longitude columns
df_geo_cleaned = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))

# Drop the latitude and longitude columns from the DataFrame
df_geo_cleaned = df_geo_cleaned.drop("latitude", "longitude")

# Convert the 'timestamp' column from a string to a timestamp data type
df_geo_cleaned = df_geo_cleaned.withColumn("timestamp", to_timestamp(col("timestamp")))

# Reorder DataFrame columns
column_order = ["ind", "country", "coordinates", "timestamp"]
df_geo_cleaned = df_geo_cleaned.select(column_order)

# Show the cleaned DataFrame
df_geo_cleaned.show()


clean the df_user DataFrame you should perform the following transformations:

Create a new column user_name that concatenates the information found in the first_name and last_name columns
Drop the first_name and last_name columns from the DataFrame
Convert the date_joined column from a string to a timestamp data type
Reorder the DataFrame columns to have the following column order:
ind
user_name
age
date_joined

In [ ]:
from pyspark.sql.functions import col, concat_ws, to_timestamp

# Create a new column 'user_name' by concatenating 'first_name' and 'last_name'
df_user_cleaned = df_user.withColumn("user_name", concat_ws(" ", col("first_name"), col("last_name")))

# Drop the 'first_name' and 'last_name' columns from the DataFrame
df_user_cleaned = df_user_cleaned.drop("first_name", "last_name")

# Convert the 'date_joined' column from a string to a timestamp data type
df_user_cleaned = df_user_cleaned.withColumn("date_joined", to_timestamp(col("date_joined")))

# Reorder DataFrame columns
column_order = ["ind", "user_name", "age", "date_joined"]
df_user_cleaned = df_user_cleaned.select(column_order)

# Show the cleaned DataFrame
df_user_cleaned.show()


Find the most popular Pinterest category people post to based on their country.


Your query should return a DataFrame that contains the following columns:

country
category
category_count, a new column containing the desired query output

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, rank
from pyspark.sql.window import Window

# Create a Spark session
spark = SparkSession.builder.appName("PinterestAnalytics").getOrCreate()

# Use window function to rank categories based on category_count
window_spec = Window.partitionBy("country").orderBy(desc("category_count"))

# Create a DataFrame with the desired query output
df_most_popular_category = df_pin_cleaned.join(
    df_geo_cleaned.select("ind", "country"),
    "ind"
).groupBy("country", "category").count() \
    .withColumn("category_count", col("count")) \
    .withColumn("rank", rank().over(window_spec)) \
    .filter(col("rank") == 1) \
    .select("country", "category", "category_count") \
    .orderBy("country")

# Show the result
df_most_popular_category.show()


Find how many posts each category had between 2018 and 2022.


Your query should return a DataFrame that contains the following columns:

post_year, a new column that contains only the year from the timestamp column
category
category_count, a new column containing the desired query output

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year

# Create a Spark session
spark = SparkSession.builder.appName("PinterestAnalytics").getOrCreate()

# Join df_geo_cleaned and df_pin_cleaned based on 'ind'
df_combined = df_geo_cleaned.join(
    df_pin_cleaned.select("ind", "category"),
    "ind"
)

# Extract the year from the timestamp column
df_posts_by_category = df_combined.withColumn("post_year", year(col("timestamp")))

# Filter posts between 2018 and 2022
df_posts_by_category = df_posts_by_category.filter((col("post_year") >= 2018) & (col("post_year") <= 2022))

# Group by post_year and category to get category counts
df_posts_by_category = df_posts_by_category.groupBy("post_year", "category").count() \
    .withColumnRenamed("count", "category_count")

# Show the result
df_posts_by_category.show()


find the user with the most follower in each country

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max, first

# Create a Spark session
spark = SparkSession.builder.appName("PinterestAnalytics").getOrCreate()

# Join df_pin_cleaned and df_geo_cleaned based on 'ind'
df_combined = df_pin_cleaned.join(
    df_geo_cleaned.select("ind", "country"),
    "ind"
)

# Step 1: For each country, find the user with the most followers
df_max_followers_per_country = df_combined.groupBy("country").agg(
    col("country"),
    first("poster_name").alias("poster_name"),  # Using first() to get any value, as you want the most followers
    max("follower_count").alias("follower_count")
)

# Show the result for Step 1
df_max_followers_per_country.show()

# Step 2: Find the country with the user having the most followers
max_follower_entry = df_max_followers_per_country.orderBy(desc("follower_count")).limit(1)

# Show the result for Step 2
max_follower_entry.show()


What is the most popular category people post to based on the following age groups:

18-24
25-35
36-50
+50
Your query should return a DataFrame that contains the following columns:

age_group, a new column based on the original age column
category

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, dense_rank, desc

# Create a Spark session
spark = SparkSession.builder.appName("PinterestAnalytics").getOrCreate()

# Create a new column 'age_group' based on the original 'age' column
df_pin_cleaned = df_pin_cleaned.join(
    df_user_cleaned.select("ind", "age"),
    "ind"
).withColumn("age_group", 
    when((col("age") >= 18) & (col("age") <= 24), "18-24")
    .when((col("age") >= 25) & (col("age") <= 35), "25-35")
    .when((col("age") >= 36) & (col("age") <= 50), "36-50")
    .otherwise("+50"))

# Group by age_group and category to get category counts
df_category_counts_by_age_group = df_pin_cleaned.groupBy("age_group", "category").count() \
    .withColumnRenamed("count", "category_count")

# Find the most popular category for each age group
df_most_popular_category_by_age_group = df_category_counts_by_age_group \
    .withColumn("rank", dense_rank().over(Window.partitionBy("age_group").orderBy(desc("category_count")))) \
    .filter(col("rank") == 1) \
    .select("age_group", "category", "category_count")

# Show the result
df_most_popular_category_by_age_group.show()


What is the median follower count for users in the following age groups:

18-24
25-35
36-50
+50
Your query should return a DataFrame that contains the following columns:

age_group, a new column based on the original age column
median_follower_count, a new column containing the desired query output

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, expr

# Create a Spark session
spark = SparkSession.builder.appName("PinterestAnalytics").getOrCreate()

# Create a new column 'age_group' based on the original 'age' column
df_pin_cleaned = df_pin_cleaned.withColumn("age_group", 
                                           when((col("age") >= 18) & (col("age") <= 24), "18-24")
                                           .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                                           .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                                           .otherwise("+50"))

# Group by age_group and calculate the median follower count using percentile_approx
df_median_follower_count_by_age_group = df_pin_cleaned.groupBy("age_group").agg(
    col("age_group"),
    expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count")
)

# Show the result
df_median_follower_count_by_age_group.show()


Find how many users have joined between 2015 and 2020.


Your query should return a DataFrame that contains the following columns:

post_year, a new column that contains only the year from the timestamp column
number_users_joined, a new column containing the desired query output

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, count

# Create a Spark session
spark = SparkSession.builder.appName("PinterestAnalytics").getOrCreate()

# Assuming you have a DataFrame df_user_cleaned with cleaned user data
# Columns: ind, user_name, date_joined

# Extract the year from the 'date_joined' column
df_user_cleaned = df_user_cleaned.withColumn("post_year", year("date_joined"))

# Filter the data for the years 2015 to 2020
df_user_joined_per_year = df_user_cleaned.filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

# Group by post_year and count the number of users joined each year
df_number_users_joined_per_year = df_user_joined_per_year.groupBy("post_year").agg(
    col("post_year"),
    count("user_name").alias("number_users_joined")
)

# Show the result
df_number_users_joined_per_year.show()


Find the median follower count of users have joined between 2015 and 2020.


Your query should return a DataFrame that contains the following columns:

post_year, a new column that contains only the year from the timestamp column
median_follower_count, a new column containing the desired query output


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, expr

# Create a Spark session
spark = SparkSession.builder.appName("PinterestAnalytics").getOrCreate()

# Extract the year from the 'timestamp' column in df_geo_cleaned
df_geo_cleaned = df_geo_cleaned.withColumn("poster_year", year("timestamp"))

df_combined = df_geo_cleaned.join(df_pin_cleaned, 'ind')

# Filter the data for the years 2015 to 2020
df_geo_joined_per_year = df_geo_cleaned.filter((col("poster_year") >= 2015) & (col("poster_year") <= 2020))

# Group by post_year and calculate the median follower count using percentile_approx
df_median_follower_count_per_year = df_combined.groupBy("poster_year").agg(
    col("poster_year"),
    expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count")
)

# Show the result
df_median_follower_count_per_year.show()

Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.


Your query should return a DataFrame that contains the following columns:

age_group, a new column based on the original age column
post_year, a new column that contains only the year from the timestamp column
median_follower_count, a new column containing the desired query output


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, expr

# Create a Spark session
spark = SparkSession.builder.appName("PinterestAnalytics").getOrCreate()

# Join df_user_cleaned, df_pin_cleaned, and df_geo_cleaned on 'ind'
df_joined = df_user_cleaned.alias("u").join(df_pin_cleaned.alias("p"), col("u.ind") == col("p.ind")).join(
    df_geo_cleaned.alias("g"), col("u.ind") == col("g.ind"))

# Extract the year from the 'timestamp' column
df_joined = df_joined.withColumn("post_year", year("g.timestamp"))

# Define age groups based on the 'age' column from df_user_cleaned
df_joined = df_joined.withColumn(
    "age_group",
    expr("CASE WHEN u.age BETWEEN 18 AND 24 THEN '18-24' "
         "WHEN u.age BETWEEN 25 AND 35 THEN '25-35' "
         "WHEN u.age BETWEEN 36 AND 50 THEN '36-50' "
         "ELSE '+50' END")
)

# Filter the data for the years 2015 to 2020
df_joined_filtered = df_joined.filter((col("poster_year") >= 2015) & (col("poster_year") <= 2020))

# Group by age_group, post_year, and calculate the median follower count using percentile_approx
df_median_follower_count_per_age_group = df_joined_filtered.groupBy("age_group", "poster_year").agg(
    expr("percentile_approx(p.follower_count, 0.5)").alias("median_follower_count")
)

# Show the result
df_median_follower_count_per_age_group.show()
